# llamaindex 加载现有向量数据库


In [1]:
import chromadb
from llama_index.core import (
    VectorStoreIndex,
    get_response_synthesizer,
    Settings,
)
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

In [2]:
# 设置嵌入模型和语言模型
Settings.embed_model = OllamaEmbedding(model_name="yxl/m3e:latest")
Settings.llm = Ollama(model="qwen2.5:0.5b", request_timeout=360)

In [3]:
# 初始化 Chroma 客户端，指定数据存储路径为当前目录下的 chroma_db 文件夹
db = chromadb.PersistentClient(path="C:/Users/Admin/Desktop/Data/diabetes/chroma_db")

# 获取或创建名为 "quickstart" 的集合，如果该集合不存在，则创建它
chroma_collection = db.get_or_create_collection("quickstart")

# 使用上述集合创建一个 ChromaVectorStore 实例，以便 llama_index 可以与 Chroma 集合进行交互
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# 创建一个存储上下文，指定向量存储为刚刚创建的 ChromaVectorStore 实例
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [4]:
# 从向量存储创建索引
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

# 配置检索器
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,  # 返回最相似的前 n 个文档片段
)

# 配置响应合成器
response_synthesizer = get_response_synthesizer()

# 组装查询引擎
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [ ]:
# 执行查询
response = query_engine.query("糖尿病如何诊断?")
print(response)  # 输出查询结果

1. 具有明确的糖尿病病史；
2. 在确诊糖尿病时或确诊之后出现的神经病变；
3. 出现神经病变的临床症状，如疼痛、麻木、感觉异常等，5项检查任意1项异常；若无临床症状，则5项检查任意2项异常也可诊断。


In [6]:
chat_engine = index.as_chat_engine()
respon = chat_engine.chat("糖尿病如何诊断")
respon.response.strip()

'我能为你提供关于糖尿病的知识，但我不能直接诊断你是否患有糖尿病或你的病情。但是，我可以告诉你一些常见的症状和指标。\n\n如果你经常出现以下症状之一或多个，你应该去看医生：\n\n1. 体重减轻：即使你的饮食没有改变，你的体重也可能减少。\n2. 感觉寒冷或饥饿：即使你正在努力保持热量平衡，这可能表示你可能需要更小心地控制自己的食物摄入量。\n3. 持续的疲劳或不适：如果你在日常活动中感到疲倦或无法集中注意力，请咨询医生。\n4. 伤口愈合缓慢：如果皮肤、黏膜或其他组织出现异常，应立即寻求医疗建议。\n\n请注意，这些症状也可能是其他健康问题的表现。最好的方法是与你的医疗保健提供者讨论任何疑虑和担忧。'